In [1]:
from gurobipy import *
import pandas as pd
import random

# 数据导入

### 模型基础数据导入
* demand: 四个需求点，demand中保存每个需求点的需求量
* capacity: 五个可选供应点，capacity中保存每个供应点的最大供给量
* fixedCosts：供应点的固定成本（供应点一旦有配货，就会产生固定成本）
* transCosts: 供应点到需求点的单位运输成本

In [2]:
demand = [15, 18, 14, 20]
capacity = [20, 22, 17, 19, 18]
fixedCosts = [12000, 15000, 17000, 13000, 16000]
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

### 为供应点与需求点建立编号，方便gurobi用向量化函数建立模型

In [3]:
plants = range(len(capacity))
warehouses = range(len(demand))

# P1：不考虑强行关闭某个供给点，让模型在求解的过程中自己选择每个供给点的开 / 关

* modelSense控制模型目标的优化方向（最大 / 最小），分别对应GRB.MAXIMIZE / MAXGRB.MINIMIZE，问题中目标是最小化成本，故选后者

In [4]:
m1 = Model("facility_p1")
m1.modelSense = GRB.MINIMIZE

### 决策变量

* open：open[p] == 1 if plant p is open.
* 函数解释：addVars(*indexes, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="")，变量类型被指定为BINARY，则lb与ub自动设置为0与1；另一种构建0-1变量的实现方式为：lb=0.0, ub=1.0, vtype=GRB.INTEGER
* addVars()返回的类型为tupledict(Gurobi tuple dict)，为GUROBI扩展的dict类，具有select() / sum()等方法，参数送入*表示针对某一维度的所有值

In [5]:
open = m1.addVars(plants, vtype=GRB.BINARY, obj=fixedCosts, name="open")

* transport：供给点配送到需求点的量，为不小于0的实数
* addVars中，前几个参数都默认为变量的下标，如add(m, n, ...），gurobi会构建一个m * n的决策变量矩阵，m与n的类型为integer / list；若设置了obj，则前面的矩阵各维度值要与obj匹配；addVars()中默认添加的变量为lb=0，ub=无穷大，类型为CONTINUOUS，所以这里不需要特别指定（建议都写一遍）

In [6]:
transport = m1.addVars(warehouses, plants, obj=transCosts, name="trans")

### 模型约束

* 供给点的总配送量不能超过其能力上限

In [7]:
m1.addConstrs((transport.sum('*',p) <= capacity[p] * open[p] for p in plants), "Capacity")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

* 需求点的总接收量要满足其需求

In [8]:
m1.addConstrs((transport.sum(w) >= demand[w] for w in warehouses), "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### 模型求解

In [9]:
try:
    m1.write('facility_p1.lp')
    %time m1.optimize()
except GurobiError as e:
    print('Error code' + str(e.errno) + ': ' + str(e))
except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))

Optimize a model with 9 rows, 25 columns and 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)

Root relaxation: objective 1.998333e+05, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 199833.333    0    1          - 199833.333      -     -    0s
H    0     0                    210500.00000 199833.333  5.07%     -    0s
     0     0 210252.941    0    2 210500.000 210252.941  0.12%     -    0s

Cutting planes:
  Gomory: 1

Explored 0 nodes (13 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 210500 
Pool ob

### 结果展示

In [10]:
print '总成本：%d' % m1.objVal
result = []
for p in plants:
    line_temp = []
    for w in warehouses:
        line_temp.append(transport[w,p].x)
    if(open[p].x > 0.5):
        line_temp.append('Open')
    else:
        line_temp.append('Closed')
    result.append(line_temp)
df = pd.DataFrame(result, index=plants, columns=warehouses + ['state'])
df

总成本：210500


,0,1,2,3,state
0,0.0,0.0,14.0,6.0,Open
1,14.0,0.0,0.0,8.0,Open
2,0.0,0.0,0.0,0.0,Closed
3,1.0,18.0,0.0,0.0,Open
4,0.0,0.0,0.0,6.0,Open


# P2：为模型送入初始值，关闭固定成本最高的供应点，求解此时最优的调度方案
* 无任何初始值设定时，问题的最优解中plant-2被关闭了，为使得效果更加直观，程序中设定固定成本最低的plant-0被关闭，求得的最优解显示结果与P1不变。
* 原因：送入初始解（或初始解的一部分），GUROBI会根据其构建一个可行解（前提是比内带的heuristic求得的初始解好），该可行解作为分支定界算法中的初始BOUND，可提高分支定界中的求解效率

* 为model设置参数的方式：m2.Params.method = 2 / m2.setParam('Method', 2)

In [11]:
m2 = Model("facility_p2")
m2.modelSense = GRB.MINIMIZE
# m2.Params.method = 2
# m2.setParam('Method', 2)

### 决策变量

In [12]:
open = m2.addVars(plants, vtype=GRB.BINARY, obj=fixedCosts, name="open")
transport = m2.addVars(warehouses, plants, obj=transCosts, name="trans")

### 模型约束
* tupledict.sum()对所有value求和，tupledict.sum(i)等价于tupledict.sum(i, '*')，表示对第i行的value求和

In [13]:
m2.addConstrs((transport.sum('*',p) <= capacity[p] * open[p] for p in plants), "Capacity")
m2.addConstrs((transport.sum(w, '*') >= demand[w] for w in warehouses), "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### 找到固定成本最低的供给点，关闭

In [14]:
for p in plants:
    open[p].start = 1.0

In [15]:
maxFixed = min(fixedCosts)
for p in plants:
    if fixedCosts[p] == maxFixed:
        open[p].start = 0.0
        print '关闭plant：%s' % p
        break

关闭plant：0


### 模型求解

In [16]:
try:
    m2.write('facility_p2.lp')
    %time m2.optimize()
except GurobiError as e:
    print('Error code' + str(e.errno) + ': ' + str(e))
except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))

Optimize a model with 9 rows, 25 columns and 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros

Loaded MIP start with objective 238400

Variable types: 20 continuous, 5 integer (5 binary)

Root relaxation: objective 1.998333e+05, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 199833.333    0    1 238400.000 199833.333  16.2%     -    0s
H    0     0                    210500.00000 199833.333  5.07%     -    0s
     0     0 210252.941    0    2 210500.000 210252.941  0.12%     -    0s

Cutting planes:
  Gomory: 1

Explored 0 nodes (13 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available proce

### 结果展示

In [17]:
print '总成本：%d' % m2.objVal
result = []
for p in plants:
    line_temp = []
    for w in warehouses:
        line_temp.append(transport[w,p].x)
    if(open[p].x > 0.5):
        line_temp.append('Open')
    else:
        line_temp.append('Closed')
    result.append(line_temp)
df = pd.DataFrame(result, index=plants, columns=warehouses + ['state'])
df

总成本：210500


,0,1,2,3,state
0,0.0,0.0,14.0,6.0,Open
1,14.0,0.0,0.0,8.0,Open
2,0.0,0.0,0.0,0.0,Closed
3,1.0,18.0,0.0,0.0,Open
4,0.0,0.0,0.0,6.0,Open


# P3：随机选择一个供应点，强制并永久让其关闭，求解关闭之后的最优的调度方案
* 强制关闭的一个供应点，需让其对应的open变量在model求解过程中时刻为0，设置方式为令其lb=ub=0即可，或添加新约束open<=0
* 关闭供应点后，应检查剩余的总供给量是否大于或等于总需求量，否则会出现程序无解

In [18]:
m3 = Model("facility_p3")
m3.modelSense = GRB.MINIMIZE

### 决策变量 & 模型约束

In [19]:
open = m3.addVars(plants, vtype=GRB.BINARY, obj=fixedCosts, name="open")
transport = m3.addVars(warehouses, plants, obj=transCosts, name="trans")

m3.addConstrs((transport.sum('*',p) <= capacity[p] * open[p] for p in plants), "Capacity")
m3.addConstrs((transport.sum(w, '*') >= demand[w] for w in warehouses), "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### 随机选择一个plant，永久关闭它

In [20]:
total_supply = 0
total_demand = sum(demand)
while(True):
    p = random.randint(0, len(plants) - 1)
    total_supply = sum([capacity[i] for i in (set(plants) - {p})])
    if(total_supply -  total_demand>= 0):
        break
print '关闭plant：%d，总供给量：%d，总需求量：%d' % (p, total_supply, total_demand)

关闭plant：0，总供给量：76，总需求量：67


In [21]:
m3.addConstr(open[p] <= 0)

<gurobi.Constr *Awaiting Model Update*>

### 模型求解

In [22]:
try:
    m3.write('facility_p3.lp')
    %time m3.optimize()
except GurobiError as e:
    print('Error code' + str(e.errno) + ': ' + str(e))
except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))

Optimize a model with 10 rows, 25 columns and 46 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 2 rows and 5 columns
Presolve time: 0.00s
Presolved: 8 rows, 20 columns, 36 nonzeros
Variable types: 16 continuous, 4 integer (4 binary)

Root relaxation: objective 2.303000e+05, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 230300.000    0    1          - 230300.000      -     -    0s
H    0     0                    238400.00000 230300.000  3.40%     -    0s
     0     0 230400.000    0    1 238400.000 230400.000  3.36%     -    0s
     0     0 230789.231    0    2 238400.000 230789.231  3.19%     -    0s
     0     0 230850.000    0    1 238400.000 230850.00

### 结果展示

In [23]:
print '总成本：%d' % m3.objVal
result = []
for p in plants:
    line_temp = []
    for w in warehouses:
        line_temp.append(transport[w,p].x)
    if(open[p].x > 0.5):
        line_temp.append('Open')
    else:
        line_temp.append('Closed')
    result.append(line_temp)
df = pd.DataFrame(result, index=plants, columns=warehouses + ['state'])
df

总成本：238400


,0,1,2,3,state
0,0.0,0.0,0.0,0.0,Closed
1,14.0,0.0,8.0,0.0,Open
2,0.0,0.0,6.0,11.0,Open
3,1.0,18.0,0.0,0.0,Open
4,0.0,0.0,0.0,9.0,Open
